In [2]:
import twitter
import pandas as pd
from collections import defaultdict
import json

ModuleNotFoundError: No module named 'twitter'

# Get tweets

In [1]:
api = twitter.Api(**open('data/twitter_key.json'))

NameError: name 'twitter' is not defined

In [3]:
#hashtag = 'DontLookGIEC'
hashtag = 'StopEACOP'

In [4]:
results = api.GetSearch(
    raw_query=f"q=%23{hashtag}%20&result_type=recent&count=100")

In [5]:
len(results)

100

In [6]:
tweet = results[0]

In [7]:
tweet.user

User(ID=710151902852476928, ScreenName=PatriciaSally_N)

In [8]:
tweet.user.followers_count

928

In [9]:
tweet.text

'RT @stopEACOP: TotalEnergies is benefiting from climate destruction and we are all paying the price.\n\nShareholders like Amundi, BNP Paribas…'

In [10]:
tweet.hashtags

[]

In [11]:
tweet.favorite_count

0

In [12]:
tweet.retweet_count

38

In [13]:
tweet.retweeted

False

In [14]:
tweet.favorited

False

# Get all tweets

In [15]:
results = api.GetSearch(
    raw_query=f"q=%23{hashtag}%20&result_type=recent&count=100")
last_tweet = results[-1]
last_tweet_id = last_tweet.id

In [16]:
last_tweet_id

1519658942963535872

In [17]:
all_tweets = results

In [18]:
while(len(results) > 1):
    results = api.GetSearch(
    raw_query=f"q=%23{hashtag}%20&result_type=recent&count=100&max_id={last_tweet_id}")
    last_tweet = results[-1]
    last_tweet_id = last_tweet.id
    all_tweets += results[1:]

In [19]:
first_tweet_id = all_tweets[0].id

In [20]:
if False:
    results = api.GetSearch(
        raw_query=f"q=%23{hashtag}%20&result_type=recent&count=100&since_id={first_tweet_id}")
    first_tweet = results[0]
    first_tweet_id = last_tweet.id
    all_tweets += results

In [21]:
len(results)

1

In [22]:
#backup = all_tweets

In [23]:
#all_tweets += results

In [24]:
len(all_tweets)

6172

In [25]:
with open(f'data/all_tweets_{hashtag.lower()}.json', 'w') as output:
    for tweet in all_tweets:
        output.write(tweet.AsJsonString() + "\n")

# Analyse

In [26]:
all_tweets = []
with open(f'data/all_tweets_{hashtag.lower()}.json') as input_file:
    for tweet_line in input_file:
        all_tweets.append(json.loads(tweet_line))

In [27]:
len(all_tweets)

6172

In [28]:
unique_ids = set([t['id'] for t in all_tweets])
len(unique_ids)

6172

In [29]:
stats = defaultdict(int)
users = defaultdict(int)
hashtags = defaultdict(int)
big_users = {}
for tweet in all_tweets:
    stats['tweets'] += 1
    if not tweet.get('retweeted_status'):
        stats['likes'] += tweet.get('favorite_count', 0)
        stats['retweets'] += tweet.get('retweet_count', 0)
    users[tweet['user']['screen_name']] += 1
    big_users[tweet['user']['screen_name']] = tweet['user'].get('followers_count', 0)
    for hastag in tweet['hashtags']:
        hashtags[hastag['text']] += 1

In [30]:
stats

defaultdict(int, {'tweets': 6172, 'likes': 15089, 'retweets': 5662})

In [31]:
print(f"""Les dernières stats:
* tweets : {stats['tweets']}
* likes : {stats['likes']}
* RT : {stats['retweets']}
""")

Les dernières stats:
* tweets : 6172
* likes : 15089
* RT : 5662



In [32]:
len(users)

2071

In [33]:
pd.Series(users).sort_values(ascending=False).head(10)

Caro_linede      145
seventdrauhl     116
princepapa1      111
ARefuelling       94
350_kenya         84
sacha_besson      75
JaneFlore81       65
TKobusiingye      58
endelstamberg     54
BxlLouise         54
dtype: int64

In [34]:
pd.Series(big_users).sort_values(ascending=False).head(10)

GretaThunberg     5065722
Greenpeace        1906645
greenpeacefr       444674
ExtinctionR        395774
350                392012
vanessa_vash       235322
WeDontHaveTime     110192
ppptec              88521
princepapa1         86947
XRebellionUK        86494
dtype: int64

In [35]:
pd.Series(hashtags).sort_values(ascending=False)

StopEACOP                 2385
EarthDayRiseupMovement     839
StopEacop                  516
EarthDay2022               492
earthdayRiseupmovement     247
                          ... 
oott                         1
Pascua                       1
greenwashing                 1
COP21                        1
JustStopOil                  1
Length: 144, dtype: int64

# Best tweets

In [46]:
best_tweets = sorted(all_tweets, key=lambda x : - x.get('retweet_count', 0) if not x.get('retweeted_status') else 0)[:10]

In [47]:
top_tweet = best_tweets[0]

In [49]:
top_tweet['text']

'Avec plusieurs associations, on interpelle @LeaSalame et @GillesBouleau pour que le climat soit au cœur du… https://t.co/38FJ1vX4Ks'

In [50]:
top_tweet['id']

1516322134053818368

In [51]:
f"https://twitter.com/{top_tweet['user']['screen_name']}/status/{top_tweet['id']}"

'https://twitter.com/ReveilEcolo/status/1516322134053818368'

In [52]:
for top_tweet in best_tweets:
    print(f"- https://twitter.com/{top_tweet['user']['screen_name']}/status/{top_tweet['id']}\n")

- https://twitter.com/ReveilEcolo/status/1516322134053818368

- https://twitter.com/cassouman40/status/1516460858846720010

- https://twitter.com/QuotaClimat/status/1514651598487752704

- https://twitter.com/QuotaClimat/status/1514967438160547845

- https://twitter.com/QuotaClimat/status/1515735675106115589

- https://twitter.com/QuotaClimat/status/1516886865613361152

- https://twitter.com/pierrecannet/status/1514989429697429505

- https://twitter.com/ClimatMedias/status/1514938305779011584

- https://twitter.com/onestpret/status/1516026324711837710

- https://twitter.com/QuotaClimat/status/1516008870547709953

